In [ ]:
import logging
from uuid import uuid4

import pandas as pd
from tqdm import tqdm

from plum_chatbot.agents.agents import invoke_chatbot
from plum_chatbot.datasources.qdrant_datasource import QdrantDatasource
from plum_chatbot.di_containers.datasources_containers import Container, container
from plum_chatbot.schemas.schema import ChatMessage, UserInput

In [ ]:
logger = logging.getLogger(__name__)
agent_id = "rag"

In [ ]:
container.init_resources()
for k, v in Container.providers.items():
    provider = v()
    if isinstance(provider, QdrantDatasource):
        provider.setup()
    logger.info(f"Dependency {k} initialized with {v}")

/plum_chatbot/src/plum_chatbot/datasources/qdrant_datasource.py:40: UserWarning: Api key is used with an insecure connection.
  self.client = QdrantClient(


In [3]:
golden_set = pd.read_csv("data/golden_set.csv")

In [ ]:
answers = []

for q in tqdm(golden_set["question"]):
    # print(q)
    user_input: UserInput = UserInput(message=q, thread_id=str(uuid4()))
    answer: ChatMessage = await invoke_chatbot(user_input, agent_id)
    answers.append(answer.content)
    # print(answer.content)
    # print()
    # print()

100%|██████████| 10/10 [00:40<00:00,  4.01s/it]


In [ ]:
golden_set["answers"] = answers
golden_set.to_csv("data/golden_set_with_answers.csv", index=False)